In [5]:
# ===========================================
# 1. 재직자 & 퇴사자 CSV 로드
# ===========================================
import pymysql
import pandas as pd
import json

# ⚠️ 경로 수정 필요!
in_path = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\백업\in_employee.csv"
out_path = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\백업\out_employee.csv"

df_in = pd.read_csv(in_path, encoding='utf-8-sig')
df_out = pd.read_csv(out_path, encoding='utf-8-sig')

# 재직자 딕셔너리: {사원명: {emailId, deptName, positionName, deptCode}}
in_dict = {}
for _, row in df_in.iterrows():
    name = str(row['사원명']).strip()
    in_dict[name] = {
        'emailId': str(row['ID']).strip() if pd.notna(row['ID']) else '',
        'deptName': str(row['부서']).strip() if pd.notna(row['부서']) else '',
        'positionName': str(row['직위']).strip() if pd.notna(row['직위']) else '',
        'deptCode': str(row['부서코드']).strip() if pd.notna(row['부서코드']) else ''
    }

# 퇴사자 딕셔너리: {사원명: {emailId, deptName, positionName, deptCode}}
out_dict = {}
for _, row in df_out.iterrows():
    name = str(row['사원명']).strip()
    out_dict[name] = {
        'emailId': str(row['ID']).strip() if pd.notna(row['ID']) else '',
        'deptName': str(row['부서']).strip() if pd.notna(row['부서']) else '',
        'positionName': str(row['직위']).strip() if pd.notna(row['직위']) else '',
        'deptCode': str(row['부서코드']).strip() if pd.notna(row['부서코드']) else ''
    }

print(f"재직자: {len(in_dict)}명")
print(f"퇴사자: {len(out_dict)}명")
print(f"재직자 샘플: {list(in_dict.items())[:2]}")
print(f"퇴사자 샘플: {list(out_dict.items())[:2]}")

재직자: 167명
퇴사자: 335명
재직자 샘플: [('외주계약(공통)', {'emailId': 'contract', 'deptName': '공통계정', 'positionName': '기타', 'deptCode': 'AB90'}), ('patent', {'emailId': 'patent', 'deptName': '공통계정', 'positionName': '기타', 'deptCode': 'AB90'})]
퇴사자 샘플: [('CEO', {'emailId': 'temp001', 'deptName': '대표이사', 'positionName': '대표이사', 'deptCode': 'AB10'}), ('PHAM NHU QUYNH', {'emailId': 'temp002', 'deptName': '글로벌사업팀', 'positionName': '책임', 'deptCode': ''})]


In [6]:
# ===========================================
# 2. documents 테이블 조회
# ===========================================
conn = pymysql.connect(
    host='localhost',
    port=3306,
    user='root',
    password='1234',
    database='any_approval',
    charset='utf8mb4'
)

query = "SELECT id, drafter_name, activities FROM documents"
df_docs = pd.read_sql(query, conn)
conn.close()

print(f"documents 총 건수: {len(df_docs)}")
print(df_docs.head())

C:\Users\LEEJUHWAN\AppData\Local\Temp\ipykernel_13996\3043956604.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_docs = pd.read_sql(query, conn)


documents 총 건수: 23320
   id drafter_name                                         activities
0   1          고상환  [{"positionName": "상무", "deptName": "ITO사업팀", ...
1   2          한승재  [{"positionName": "", "deptName": "", "actionL...
2   3          김선홍  [{"positionName": "선임", "deptName": "공공사업팀", "...
3   4          이필호  [{"positionName": "", "deptName": "", "actionL...
4   5          한승재  [{"positionName": "", "deptName": "", "actionL...


In [7]:
# ===========================================
# 3. 업데이트 데이터 생성
# ===========================================
def get_person_info(name):
    """사람 정보 반환 (재직자 우선, 그다음 퇴사자)"""
    name = str(name).strip() if name else ''
    
    # 1순위: 재직자
    if name in in_dict:
        info = in_dict[name]
        return {
            'email': info['emailId'],
            'position': info['positionName'],
            'dept': info['deptName'],
            'dept_code': info['deptCode'],
            'source': 'in'
        }
    
    # 2순위: 퇴사자
    if name in out_dict:
        info = out_dict[name]
        return {
            'email': info['emailId'],
            'position': info['positionName'],
            'dept': info['deptName'],
            'dept_code': info['deptCode'],
            'source': 'out'
        }
    
    # 둘 다 없으면 (미등록자) - 이름은 유지, 나머지 공란
    return {
        'email': '',
        'position': '',
        'dept': '',
        'dept_code': '',
        'source': 'unknown'
    }


def process_activities(activities_str):
    """activities JSON 처리"""
    if not activities_str or pd.isna(activities_str):
        return None
    
    try:
        activities = json.loads(activities_str)
    except json.JSONDecodeError:
        return None
    
    for activity in activities:
        name = activity.get('name', '').strip()
        info = get_person_info(name)
        
        activity['emailId'] = info['email']
        activity['positionName'] = info['position']
        activity['deptName'] = info['dept']
        activity['deptCode'] = info['dept_code']
    
    return json.dumps(activities, ensure_ascii=False)


# 업데이트 데이터 생성
update_data = []
unknown_drafters = []

for idx, row in df_docs.iterrows():
    drafter_info = get_person_info(row['drafter_name'])
    new_activities = process_activities(row['activities'])
    
    if drafter_info['source'] == 'unknown' and row['drafter_name']:
        unknown_drafters.append(row['drafter_name'])
    
    update_data.append({
        'id': row['id'],
        'drafter_email': drafter_info['email'],
        'drafter_position': drafter_info['position'],
        'drafter_dept': drafter_info['dept'],
        'drafter_dept_code': drafter_info['dept_code'],
        'activities': new_activities
    })

df_update = pd.DataFrame(update_data)
print(f"업데이트 대상: {len(df_update)}건")
print(df_update.head(10))

# 미등록자 확인
unknown_unique = list(set(unknown_drafters))
print(f"\n⚠️ 재직자/퇴사자 목록 둘 다 없는 사람: {len(unknown_unique)}명")
if unknown_unique:
    print(unknown_unique[:20])

업데이트 대상: 23320건
   id drafter_email drafter_position drafter_dept drafter_dept_code  \
0   1          shko               상무       ITO사업팀              DB60   
1   2       temp323               선임       사업지원파트                     
2   3         shkim               선임    KAPE Part              DB12   
3   4       temp252               책임        ETRI팀                     
4   5       temp323               선임       사업지원파트                     
5   6       temp323               선임       사업지원파트                     
6   7        shshin               책임       IP서비스팀              GB30   
7   8       temp323               선임       사업지원파트                     
8   9       temp323               선임       사업지원파트                     
9  10       temp323               선임       사업지원파트                     

                                          activities  
0  [{"positionName": "상무", "deptName": "ITO사업팀", ...  
1  [{"positionName": "선임", "deptName": "사업지원파트", ...  
2  [{"positionName": "선임", "deptName"

In [8]:
# ===========================================
# 4. 검증
# ===========================================
print("=" * 50)
print("검증")
print("=" * 50)

# 재직자 drafter 통계
in_emails = set(v['emailId'] for v in in_dict.values() if v['emailId'])
in_count = df_update[df_update['drafter_email'].isin(in_emails)].shape[0]
print(f"재직자 drafter 문서: {in_count}건")

# 퇴사자 drafter 통계
out_emails = set(v['emailId'] for v in out_dict.values() if v['emailId'])
out_count = df_update[df_update['drafter_email'].isin(out_emails)].shape[0]
print(f"퇴사자 drafter 문서: {out_count}건")

# 빈 email (미등록자)
empty_count = df_update[df_update['drafter_email'] == ''].shape[0]
print(f"미등록자 drafter 문서 (email 공란): {empty_count}건")

print(f"\n합계: {in_count + out_count + empty_count}건 (전체: {len(df_update)}건)")

검증
재직자 drafter 문서: 12291건
퇴사자 drafter 문서: 11029건
미등록자 drafter 문서 (email 공란): 0건

합계: 23320건 (전체: 23320건)


In [9]:
# ===========================================
# 5. 샘플 확인
# ===========================================
print("[재직자 샘플]")
in_emails = set(v['emailId'] for v in in_dict.values() if v['emailId'] and not v['emailId'].startswith('temp'))
sample_in = df_update[df_update['drafter_email'].isin(in_emails)].head(5)
print(sample_in[['id', 'drafter_email', 'drafter_dept', 'drafter_position', 'drafter_dept_code']])

print("\n[퇴사자 샘플]")
sample_out = df_update[df_update['drafter_email'].str.startswith('temp', na=False)].head(5)
print(sample_out[['id', 'drafter_email', 'drafter_dept', 'drafter_position', 'drafter_dept_code']])

print("\n[미등록자 샘플]")
sample_unknown = df_update[df_update['drafter_email'] == ''].head(5)
print(sample_unknown[['id', 'drafter_email', 'drafter_dept', 'drafter_position', 'drafter_dept_code']])

[재직자 샘플]
    id drafter_email drafter_dept drafter_position drafter_dept_code
0    1          shko       ITO사업팀               상무              DB60
2    3         shkim    KAPE Part               선임              DB12
6    7        shshin       IP서비스팀               책임              GB30
14  15        cspark       AI사업본부               책임              RB00
17  18      biilykim       IP솔루션팀               책임              GB20

[퇴사자 샘플]
   id drafter_email drafter_dept drafter_position drafter_dept_code
1   2       temp323       사업지원파트               선임                  
3   4       temp252        ETRI팀               책임                  
4   5       temp323       사업지원파트               선임                  
5   6       temp323       사업지원파트               선임                  
7   8       temp323       사업지원파트               선임                  

[미등록자 샘플]
Empty DataFrame
Columns: [id, drafter_email, drafter_dept, drafter_position, drafter_dept_code]
Index: []


In [10]:
# ===========================================
# 6. 실제 DB 업데이트 (확인 후 실행!)
# ===========================================
# ⚠️ 위 검증 결과 확인 후 실행!

conn = pymysql.connect(
    host='localhost',
    port=3306,
    user='root',
    password='1234',
    database='any_approval',
    charset='utf8mb4'
)
cursor = conn.cursor()

update_count = 0
error_count = 0

for idx, row in df_update.iterrows():
    try:
        query = """UPDATE documents 
        SET drafter_email = %s, 
            drafter_position = %s, 
            drafter_dept = %s, 
            drafter_dept_code = %s,
            activities = %s
        WHERE id = %s"""
        
        cursor.execute(query, (
            row['drafter_email'],
            row['drafter_position'],
            row['drafter_dept'],
            row['drafter_dept_code'],
            row['activities'],
            row['id']
        ))
        update_count += 1
        
        if update_count % 1000 == 0:
            print(f"진행 중... {update_count}/{len(df_update)}")
            
    except Exception as e:
        error_count += 1
        print(f"에러 - ID {row['id']}: {e}")

conn.commit()
cursor.close()
conn.close()

print(f"\n완료! 업데이트: {update_count}건, 에러: {error_count}건")

진행 중... 1000/23320
진행 중... 2000/23320
진행 중... 3000/23320
진행 중... 4000/23320
진행 중... 5000/23320
진행 중... 6000/23320
진행 중... 7000/23320
진행 중... 8000/23320
진행 중... 9000/23320
진행 중... 10000/23320
진행 중... 11000/23320
진행 중... 12000/23320
진행 중... 13000/23320
진행 중... 14000/23320
진행 중... 15000/23320
진행 중... 16000/23320
진행 중... 17000/23320
진행 중... 18000/23320
진행 중... 19000/23320
진행 중... 20000/23320
진행 중... 21000/23320
진행 중... 22000/23320
진행 중... 23000/23320

완료! 업데이트: 23320건, 에러: 0건
